In [1]:
!git clone https://github.com/Teddy-XiongGZ/MedRAG

Cloning into 'MedRAG'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 143 (delta 44), reused 29 (delta 29), pack-reused 89 (from 2)
Receiving objects: 100% (143/143), 155.03 KiB | 7.75 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [4]:
!pip install -q \
    faiss-cpu==1.7.4 \
    python-liquid==1.10.2 \
    openai==0.28.0 \
    langchain==0.0.345 \
    tiktoken==0.5.1 \
    datasets==2.16.1 \
    transformers \
    sentence-transformers \
    accelerate \
    huggingface-hub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
google-genai 1.21.1 requires anyio<5.0.0,>=4.8.0, but you have anyio 3.7.1 which is incompatible.
langchain-text-splitters 0.3.8 requires langchain-core<1.0.0,>=0.3.51, but you have langchain-core 0.0.13 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.10.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
dataproc-spar

In [7]:
# Cell 3: Set up environment and API Key
import os
import sys
from kaggle_secrets import UserSecretsClient

# Add the cloned repository's source to the Python path
sys.path.append('/kaggle/working/MedRAG')

# Securely get the API key
user_secrets = UserSecretsClient()
os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_API_KEY")

# Now you can import the MedRAG library
from src.medrag import MedRAG

print("Setup complete. MedRAG is ready to use.")

2025-09-21 09:15:13.308028: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758446113.708624      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758446113.825155      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Setup complete. MedRAG is ready to use.


In [16]:
# Cell 4: Define the question and run the examples

# The medical question from the example
question = "A lesion causing compression of the facial nerve at the stylomastoid foramen will cause ipsilateral"
options = {
    "A": "paralysis of the facial muscles.",
    "B": "paralysis of the facial muscles and loss of taste.",
    "C": "paralysis of the facial muscles, loss of taste and lacrimation.",
    "D": "paralysis of the facial muscles, loss of taste, lacrimation and decreased salivation."
}

# --- Example 1: Chain of Thought (CoT) ---
print("--- Running Chain of Thought (CoT) ---")
cot = MedRAG(llm_name="OpenAI/gpt-3.5-turbo-16k", rag=False)
answer, _, _ = cot.answer(question=question, options=options)
print(f"CoT Final answer in json with rationale: {answer}\n")
# The first time you run this, it will download the MedCPT model. This can take a few minutes.

# --- Example 2: Standard MedRAG ---
print("--- Running Standard MedRAG ---")
medrag = MedRAG(llm_name="OpenAI/gpt-3.5-turbo-16k", rag=True, retriever_name="MedCPT", corpus_name="Textbooks")
answer, snippets, scores = medrag.answer(question=question, options=options, k=32)
print(f"MedRAG Final answer in json with rationale: {answer}\n")


# --- Example 3: i-MedRAG (with follow-up questions) ---
print("--- Running i-MedRAG (Iterative Follow-up) ---")
# Using corpus_cache=True will speed up subsequent runs if you were doing many in a row
imedrag = MedRAG(llm_name="OpenAI/gpt-3.5-turbo-16k", rag=True, follow_up=True, retriever_name="MedCPT", corpus_name="Textbooks", corpus_cache=True)
answer, history = imedrag.answer(question=question, options=options, k=32, n_rounds=2, n_queries=3) # Using fewer rounds for a quicker demo
print(f"i-MedRAG Final answer in json: {answer}")
print("\n--- i-MedRAG Analysis and Follow-up Queries ---")
print(f"Raw answer with analysis: {history[-3]}")
print(f"Follow-up queries generated: {[item.split('Answer: ')[0].strip() for item in history[-4]['content'].split('Query: ')[1:]]}")

--- Running Chain of Thought (CoT) ---


InvalidRequestError: Unsupported OpenAI-Version header provided: 2023-07-01-preview. (HINT: you can provide any of the following supported versions: 2020-10-01, 2020-11-07. Alternatively, you can simply omit this header to use the default version associated with your account.)